# Analisis y Buscador: Catalogo de Codigos Postales de Mexico
**Autor:** Francisco Solis Pedraza  
**Tecnologias:** Python, PyMySQL, SQLAlchemy, Pandas, Unidecode.

Este notebook integra la exploracion, limpieza, ingesta y consulta avanzada del Catalogo Nacional de Codigos Postales de SEPOMEX.

---

## 1. Importacion de Librerias y Configuracion

In [ ]:
import pandas as pd
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine, text
from unidecode import unidecode

## 1. Conexion e Ingesta a MariaDB

In [ ]:
# 1. CONFIGURACIÓN DE ENTORNO Y CONEXIÓN
load_dotenv()

user = os.getenv("DB_USER")
password = os.getenv("DB_PASS")
host = os.getenv("DB_HOST", "localhost")
port = os.getenv("DB_PORT", "3306")
db_name = os.getenv("DB_NAME", "correos_de_mexico")

# Crear engine con el driver PyMySQL
conexion_url = f"mysql+pymysql://{user}:{password}@{host}:{port}/{db_name}"
engine = create_engine(conexion_url)

# 2. EXTRACCIÓN DE DATOS DESDE LA BASE DE DATOS
try:
    query_full = "SELECT * FROM codigos_postales"
    data = pd.read_sql(query_full, con=engine)
    print(f"Éxito: Se extrajeron {len(data)} registros desde MariaDB.")
except Exception as e:
    print(f"Error: No se pudo leer la tabla. ¿Ya ejecutaste el script .sql en DBeaver? \nDetalle: {e}")
    data = None

Éxito: Se extrajeron 155857 registros desde MariaDB.


---

## 4. Consultas Analiticas (SQL)

### 4.1 Conteo de ciudades unicas en Mexico

In [ ]:
consulta_1 = "SELECT COUNT(DISTINCT d_ciudad) AS total_ciudades FROM codigos_postales;"
df1 = pd.read_sql(consulta_1, con=engine)
print(df1)

   total_ciudades
0             644


### 4.2 Rango valido de Codigos Postales

In [ ]:
consulta_2 = "SELECT MIN(d_CP) AS cp_minimo, MAX(d_CP) AS cp_maximo FROM codigos_postales;"
df2 = pd.read_sql(consulta_2, con=engine)
print(df2)

   cp_minimo  cp_maximo
0       1001      99961


### 4.3 Asentamientos que incluyen "Maria"
Utilizamos parametros seguros para evitar inyeccion SQL y manejar caracteres especiales.

In [ ]:
palabra = 'Maria'
consulta_3 = text("""
    SELECT * FROM codigos_postales 
    WHERE d_asenta LIKE :p OR d_asenta LIKE :p2 OR d_asenta LIKE :p3
""")
df3 = pd.read_sql(consulta_3, con=engine, params={'p': f'%{palabra}%', 'p2': f'{palabra}%', 'p3': f'%{palabra}'})
print(f"Encontrados {len(df3)} asentamientos.")

Encontrados 1485 asentamientos.


### 4.4 Distribucion de zonas (Urbano vs Rural)

In [ ]:
consulta_4 = "SELECT d_zona, COUNT(*) AS total FROM codigos_postales GROUP BY d_zona;"
df4 = pd.read_sql(consulta_4, con=engine)
print(df4)

       d_zona  total
0       Rural  92091
1  Semiurbano    559
2      Urbano  63207


---

## 5. Buscador de Direcciones Optimizado

Preparamos una vista simplificada y normalizada para busquedas interactivas de usuario.

In [ ]:
# Limpieza de columnas y renombre para el buscador
remove_labels = ['d_tipo_asenta', 'd_CP', 'c_estado', 'c_oficina', 'c_CP', 'c_tipo_asenta',
                 'c_mnpio', 'id_asenta_cpcons', 'd_zona', 'c_cve_ciudad']

data_busqueda = data.drop(remove_labels, axis=1)
data_busqueda.columns = ["CP", "ASENTAMIENTO", "MUNICIPIO", "ESTADO", "CIUDAD"]

# Reordenar y normalizar texto (Quitar acentos y mayusculas)
data_busqueda = data_busqueda[["ASENTAMIENTO", "MUNICIPIO", "CIUDAD", "ESTADO", "CP"]]

for col in ["ASENTAMIENTO", "MUNICIPIO", "CIUDAD", "ESTADO"]:
    data_busqueda[col] = data_busqueda[col].fillna('').astype(str).str.upper().apply(unidecode)

print("Sistema de busqueda listo.")

Sistema de busqueda listo.


### 5.1 Interfaz de Usuario

In [ ]:
def buscador_interactivo():
    print("\n--- Buscador de Direcciones ---")
    asentamiento = input("Ingrese colonia (o Enter para saltar): ").upper().strip()
    municipio = input("Ingrese municipio (o Enter para saltar): ").upper().strip()
    
    filtro = data_busqueda.copy()
    
    if asentamiento:
        filtro = filtro[filtro["ASENTAMIENTO"].str.contains(unidecode(asentamiento))]
    if municipio:
        filtro = filtro[filtro["MUNICIPIO"].str.contains(unidecode(municipio))]
        
    if filtro.empty:
        print("No se encontraron coincidencias.")
    else:
        print(f"\nResultados encontrados:\n{filtro.head(10)}")

# Ejecutar si es necesario
# buscador_interactivo()